In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
df = spark.read.options(header = True, infer_schema = True).csv("/FileStore/tables/*")

In [0]:
df.columns

Out[5]: ['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [0]:
df.show(10)

+-----+------+---------+---------+---------+----+----+------+
|EMPNO| ENAME|      JOB|      MGR| HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+---------+---------+----+----+------+
| 7369| SMITH|CLERK7902|17-DEC-80|      800|null|  20|  null|
| 7499| ALLEN| SALESMAN|     7698|20-FEB-81|1600| 300|    30|
| 7521|  WARD| SALESMAN|     7698|22-FEB-81|1250| 500|    30|
| 7566| JONES|  MANAGER|     7839|02-APR-81|2975|  20|  null|
| 7654|MARTIN| SALESMAN|     7698|28-SEP-81|1250|1400|    30|
| 7698| BLAKE|  MANAGER|     7839|01-MAY-81|2850|null|    30|
| 7782| CLARK|  MANAGER|     7839|09-JUN-81|2450|null|    10|
| 7788| SCOTT|  ANALYST|     7566|19-APR-87|3000|null|    20|
| 7839|  KING|PRESIDENT|     null|17-NOV-81|5000|null|    10|
| 7844|TURNER| SALESMAN|     7698|08-SEP-81|1500|   0|    30|
+-----+------+---------+---------+---------+----+----+------+
only showing top 10 rows



Dropping duplicates

In [0]:
df.count()

Out[7]: 28

In [0]:
df1 = df.dropDuplicates()
df1.count()

Out[8]: 14

In [0]:
df1.show()

+-----+------+---------+---------+---------+----+----+------+
|EMPNO| ENAME|      JOB|      MGR| HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+---------+---------+----+----+------+
| 7499| ALLEN| SALESMAN|     7698|20-FEB-81|1600| 300|    30|
| 7902|  FORD|  ANALYST|     7566|03-DEC-81|3000|  20|  null|
| 7521|  WARD| SALESMAN|     7698|22-FEB-81|1250| 500|    30|
| 7934|MILLER|    CLERK|     7782|23-JAN-82|1300|  10|  null|
| 7900| JAMES|    CLERK|     7698|03-DEC-81| 950|  30|  null|
| 7844|TURNER| SALESMAN|     7698|08-SEP-81|1500|   0|    30|
| 7782| CLARK|  MANAGER|     7839|09-JUN-81|2450|null|    10|
| 7698| BLAKE|  MANAGER|     7839|01-MAY-81|2850|null|    30|
| 7876| ADAMS|    CLERK|     7788|23-MAY-87|1100|  20|  null|
| 7566| JONES|  MANAGER|     7839|02-APR-81|2975|  20|  null|
| 7369| SMITH|CLERK7902|17-DEC-80|      800|null|  20|  null|
| 7839|  KING|PRESIDENT|     null|17-NOV-81|5000|null|    10|
| 7788| SCOTT|  ANALYST|     7566|19-APR-87|3000|null|    20|
| 7654|M

We observe that a missing comma has distorted one of the records

In [0]:
from pyspark.sql.functions import when, regexp_replace

In [0]:
df2 = df1.withColumn("JOB", when(df1.SAL.isNull(),regexp_replace(df1.JOB, "CLERK7902","CLERK")).otherwise(df1.JOB)).withColumn("MGR", when(df1.SAL.isNull(), regexp_replace(df1.MGR, "17-DEC-80","7902")).otherwise(df1.MGR)).withColumn("HIREDATE", when(df1.SAL.isNull(), regexp_replace(df1.HIREDATE, "800","17-DEC-80")).otherwise(df1.HIREDATE)).withColumn("COMM", when(df1.SAL.isNull(), regexp_replace(df1.COMM, "20",None)).otherwise(df1.COMM)).withColumn("DEPTNO", when(df1.SAL.isNull(), "20").otherwise(df1.DEPTNO)).withColumn("SAL", when(df1.SAL.isNull(), "800").otherwise(df1.SAL))

In [0]:
df2.show()

+-----+------+---------+----+---------+----+----+------+
|EMPNO| ENAME|      JOB| MGR| HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+---------+----+----+------+
| 7499| ALLEN| SALESMAN|7698|20-FEB-81|1600| 300|    30|
| 7902|  FORD|  ANALYST|7566|03-DEC-81|3000|  20|  null|
| 7521|  WARD| SALESMAN|7698|22-FEB-81|1250| 500|    30|
| 7934|MILLER|    CLERK|7782|23-JAN-82|1300|  10|  null|
| 7900| JAMES|    CLERK|7698|03-DEC-81| 950|  30|  null|
| 7844|TURNER| SALESMAN|7698|08-SEP-81|1500|   0|    30|
| 7782| CLARK|  MANAGER|7839|09-JUN-81|2450|null|    10|
| 7698| BLAKE|  MANAGER|7839|01-MAY-81|2850|null|    30|
| 7876| ADAMS|    CLERK|7788|23-MAY-87|1100|  20|  null|
| 7566| JONES|  MANAGER|7839|02-APR-81|2975|  20|  null|
| 7369| SMITH|    CLERK|7902|17-DEC-80| 800|null|    20|
| 7839|  KING|PRESIDENT|null|17-NOV-81|5000|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-APR-87|3000|null|    20|
| 7654|MARTIN| SALESMAN|7698|28-SEP-81|1250|1400|    30|
+-----+------+---------+----+--

Converting attributes to proper datatypes

In [0]:
df2.dtypes

Out[13]: [('EMPNO', 'string'),
 ('ENAME', 'string'),
 ('JOB', 'string'),
 ('MGR', 'string'),
 ('HIREDATE', 'string'),
 ('SAL', 'string'),
 ('COMM', 'string'),
 ('DEPTNO', 'string')]

In [0]:
columns = ["EMPNO", "MGR", "SAL", "COMM", "DEPTNO"]

In [0]:
from pyspark.sql.functions import to_date

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [0]:
df3 = df2.select(*(
                         col(column) for column in df1.columns if column not in columns
                     ),
                     *(
                         col(column).cast(IntegerType()) for column in columns
                     )
                    )

In [0]:
df3.dtypes

Out[18]: [('ENAME', 'string'),
 ('JOB', 'string'),
 ('HIREDATE', 'string'),
 ('EMPNO', 'int'),
 ('MGR', 'int'),
 ('SAL', 'int'),
 ('COMM', 'int'),
 ('DEPTNO', 'int')]

In [0]:
df3.show(14)

+------+---------+---------+-----+----+----+----+------+
| ENAME|      JOB| HIREDATE|EMPNO| MGR| SAL|COMM|DEPTNO|
+------+---------+---------+-----+----+----+----+------+
| ALLEN| SALESMAN|20-FEB-81| 7499|7698|1600| 300|    30|
|  FORD|  ANALYST|03-DEC-81| 7902|7566|3000|  20|  null|
|  WARD| SALESMAN|22-FEB-81| 7521|7698|1250| 500|    30|
|MILLER|    CLERK|23-JAN-82| 7934|7782|1300|  10|  null|
| JAMES|    CLERK|03-DEC-81| 7900|7698| 950|  30|  null|
|TURNER| SALESMAN|08-SEP-81| 7844|7698|1500|   0|    30|
| CLARK|  MANAGER|09-JUN-81| 7782|7839|2450|null|    10|
| BLAKE|  MANAGER|01-MAY-81| 7698|7839|2850|null|    30|
| ADAMS|    CLERK|23-MAY-87| 7876|7788|1100|  20|  null|
| JONES|  MANAGER|02-APR-81| 7566|7839|2975|  20|  null|
| SMITH|    CLERK|17-DEC-80| 7369|7902| 800|null|    20|
|  KING|PRESIDENT|17-NOV-81| 7839|null|5000|null|    10|
| SCOTT|  ANALYST|19-APR-87| 7788|7566|3000|null|    20|
|MARTIN| SALESMAN|28-SEP-81| 7654|7698|1250|1400|    30|
+------+---------+---------+---

There are certain departments with no department number; categorizing them as 1000

In [0]:
df4 = df3.na.fill(value = 1000, subset = ["DEPTNO"])

In [0]:
df4.show()

+------+---------+---------+-----+----+----+----+------+
| ENAME|      JOB| HIREDATE|EMPNO| MGR| SAL|COMM|DEPTNO|
+------+---------+---------+-----+----+----+----+------+
| ALLEN| SALESMAN|20-FEB-81| 7499|7698|1600| 300|    30|
|  FORD|  ANALYST|03-DEC-81| 7902|7566|3000|  20|  1000|
|  WARD| SALESMAN|22-FEB-81| 7521|7698|1250| 500|    30|
|MILLER|    CLERK|23-JAN-82| 7934|7782|1300|  10|  1000|
| JAMES|    CLERK|03-DEC-81| 7900|7698| 950|  30|  1000|
|TURNER| SALESMAN|08-SEP-81| 7844|7698|1500|   0|    30|
| CLARK|  MANAGER|09-JUN-81| 7782|7839|2450|null|    10|
| BLAKE|  MANAGER|01-MAY-81| 7698|7839|2850|null|    30|
| ADAMS|    CLERK|23-MAY-87| 7876|7788|1100|  20|  1000|
| JONES|  MANAGER|02-APR-81| 7566|7839|2975|  20|  1000|
| SMITH|    CLERK|17-DEC-80| 7369|7902| 800|null|    20|
|  KING|PRESIDENT|17-NOV-81| 7839|null|5000|null|    10|
| SCOTT|  ANALYST|19-APR-87| 7788|7566|3000|null|    20|
|MARTIN| SALESMAN|28-SEP-81| 7654|7698|1250|1400|    30|
+------+---------+---------+---

FILLING NULL VALUES FOR COMMISSION AS ZERO

In [0]:
df5 = df4.na.fill(value = 0, subset= ["COMM"])

In [0]:
df5.show(14)

+------+---------+---------+-----+----+----+----+------+
| ENAME|      JOB| HIREDATE|EMPNO| MGR| SAL|COMM|DEPTNO|
+------+---------+---------+-----+----+----+----+------+
| ALLEN| SALESMAN|20-FEB-81| 7499|7698|1600| 300|    30|
|  FORD|  ANALYST|03-DEC-81| 7902|7566|3000|  20|  1000|
|  WARD| SALESMAN|22-FEB-81| 7521|7698|1250| 500|    30|
|MILLER|    CLERK|23-JAN-82| 7934|7782|1300|  10|  1000|
| JAMES|    CLERK|03-DEC-81| 7900|7698| 950|  30|  1000|
|TURNER| SALESMAN|08-SEP-81| 7844|7698|1500|   0|    30|
| CLARK|  MANAGER|09-JUN-81| 7782|7839|2450|   0|    10|
| BLAKE|  MANAGER|01-MAY-81| 7698|7839|2850|   0|    30|
| ADAMS|    CLERK|23-MAY-87| 7876|7788|1100|  20|  1000|
| JONES|  MANAGER|02-APR-81| 7566|7839|2975|  20|  1000|
| SMITH|    CLERK|17-DEC-80| 7369|7902| 800|   0|    20|
|  KING|PRESIDENT|17-NOV-81| 7839|null|5000|   0|    10|
| SCOTT|  ANALYST|19-APR-87| 7788|7566|3000|   0|    20|
|MARTIN| SALESMAN|28-SEP-81| 7654|7698|1250|1400|    30|
+------+---------+---------+---

In [0]:
df5.dtypes

Out[24]: [('ENAME', 'string'),
 ('JOB', 'string'),
 ('HIREDATE', 'string'),
 ('EMPNO', 'int'),
 ('MGR', 'int'),
 ('SAL', 'int'),
 ('COMM', 'int'),
 ('DEPTNO', 'int')]

Converting HIREDATE attribute to Datetype

In [0]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import *

In [0]:
df6 = df5.select(*
                (
                    col(column) for column in df5.columns if column != "HIREDATE"
                ),*
                 (
                    add_months(to_date(col(column), "dd-MMM-yy"), - 12*100).alias("HIREDATE") for column in df5.columns if column == "HIREDATE"
                )
                )

In [0]:
df6.show(14)

+------+---------+-----+----+----+----+------+----------+
| ENAME|      JOB|EMPNO| MGR| SAL|COMM|DEPTNO|  HIREDATE|
+------+---------+-----+----+----+----+------+----------+
| ALLEN| SALESMAN| 7499|7698|1600| 300|    30|1981-02-20|
|  FORD|  ANALYST| 7902|7566|3000|  20|  1000|1981-12-03|
|  WARD| SALESMAN| 7521|7698|1250| 500|    30|1981-02-22|
|MILLER|    CLERK| 7934|7782|1300|  10|  1000|1982-01-23|
| JAMES|    CLERK| 7900|7698| 950|  30|  1000|1981-12-03|
|TURNER| SALESMAN| 7844|7698|1500|   0|    30|1981-09-08|
| CLARK|  MANAGER| 7782|7839|2450|   0|    10|1981-06-09|
| BLAKE|  MANAGER| 7698|7839|2850|   0|    30|1981-05-01|
| ADAMS|    CLERK| 7876|7788|1100|  20|  1000|1987-05-23|
| JONES|  MANAGER| 7566|7839|2975|  20|  1000|1981-04-02|
| SMITH|    CLERK| 7369|7902| 800|   0|    20|1980-12-17|
|  KING|PRESIDENT| 7839|null|5000|   0|    10|1981-11-17|
| SCOTT|  ANALYST| 7788|7566|3000|   0|    20|1987-04-19|
|MARTIN| SALESMAN| 7654|7698|1250|1400|    30|1981-09-28|
+------+------

SUM OF  SALARIES OF ALL THE DEPARTMENTS

In [0]:
df7 = df6.withColumn("total_salary" , col("SAL") + col("COMM"))

In [0]:
df7.show()

+------+---------+-----+----+----+----+------+----------+------------+
| ENAME|      JOB|EMPNO| MGR| SAL|COMM|DEPTNO|  HIREDATE|total_salary|
+------+---------+-----+----+----+----+------+----------+------------+
| ALLEN| SALESMAN| 7499|7698|1600| 300|    30|1981-02-20|        1900|
|  FORD|  ANALYST| 7902|7566|3000|  20|  1000|1981-12-03|        3020|
|  WARD| SALESMAN| 7521|7698|1250| 500|    30|1981-02-22|        1750|
|MILLER|    CLERK| 7934|7782|1300|  10|  1000|1982-01-23|        1310|
| JAMES|    CLERK| 7900|7698| 950|  30|  1000|1981-12-03|         980|
|TURNER| SALESMAN| 7844|7698|1500|   0|    30|1981-09-08|        1500|
| CLARK|  MANAGER| 7782|7839|2450|   0|    10|1981-06-09|        2450|
| BLAKE|  MANAGER| 7698|7839|2850|   0|    30|1981-05-01|        2850|
| ADAMS|    CLERK| 7876|7788|1100|  20|  1000|1987-05-23|        1120|
| JONES|  MANAGER| 7566|7839|2975|  20|  1000|1981-04-02|        2995|
| SMITH|    CLERK| 7369|7902| 800|   0|    20|1980-12-17|         800|
|  KIN

In [0]:
df7.groupBy("DEPTNO").agg(sum(df7.total_salary)).show()

+------+-----------------+
|DEPTNO|sum(total_salary)|
+------+-----------------+
|    20|             3800|
|  1000|             9425|
|    10|             7450|
|    30|            10650|
+------+-----------------+



Max salary dept wise

In [0]:
df7.groupBy("DEPTNO").agg(max(df7.total_salary)).show()

+------+-----------------+
|DEPTNO|max(total_salary)|
+------+-----------------+
|    20|             3000|
|  1000|             3020|
|    10|             5000|
|    30|             2850|
+------+-----------------+



Min salary dept wise

In [0]:
df7.groupBy("DEPTNO").agg(min(df7.total_salary)).show()

+------+-----------------+
|DEPTNO|min(total_salary)|
+------+-----------------+
|    20|              800|
|  1000|              980|
|    10|             2450|
|    30|             1500|
+------+-----------------+



Average salary dept wise

In [0]:
df7.groupBy("DEPTNO").agg(avg(df7.total_salary)).show()

+------+-----------------+
|DEPTNO|avg(total_salary)|
+------+-----------------+
|    20|           1900.0|
|  1000|           1885.0|
|    10|           3725.0|
|    30|           2130.0|
+------+-----------------+



sorting the data based on salary and department in ascending and descending order respectively.

In [0]:
df7.sort(df7.total_salary.asc(), df7.DEPTNO.desc()).show()

+------+---------+-----+----+----+----+------+----------+------------+
| ENAME|      JOB|EMPNO| MGR| SAL|COMM|DEPTNO|  HIREDATE|total_salary|
+------+---------+-----+----+----+----+------+----------+------------+
| SMITH|    CLERK| 7369|7902| 800|   0|    20|1980-12-17|         800|
| JAMES|    CLERK| 7900|7698| 950|  30|  1000|1981-12-03|         980|
| ADAMS|    CLERK| 7876|7788|1100|  20|  1000|1987-05-23|        1120|
|MILLER|    CLERK| 7934|7782|1300|  10|  1000|1982-01-23|        1310|
|TURNER| SALESMAN| 7844|7698|1500|   0|    30|1981-09-08|        1500|
|  WARD| SALESMAN| 7521|7698|1250| 500|    30|1981-02-22|        1750|
| ALLEN| SALESMAN| 7499|7698|1600| 300|    30|1981-02-20|        1900|
| CLARK|  MANAGER| 7782|7839|2450|   0|    10|1981-06-09|        2450|
|MARTIN| SALESMAN| 7654|7698|1250|1400|    30|1981-09-28|        2650|
| BLAKE|  MANAGER| 7698|7839|2850|   0|    30|1981-05-01|        2850|
| JONES|  MANAGER| 7566|7839|2975|  20|  1000|1981-04-02|        2995|
| SCOT

Create a dataframe of employee with id, name and manager_id and dataframe of manager with manager_id and manager_name and perform left outer join operation.

In [0]:
df8 = df7.select("EMPNO", "ENAME", "MGR")

In [0]:
df8.show()

+-----+------+----+
|EMPNO| ENAME| MGR|
+-----+------+----+
| 7499| ALLEN|7698|
| 7902|  FORD|7566|
| 7521|  WARD|7698|
| 7934|MILLER|7782|
| 7900| JAMES|7698|
| 7844|TURNER|7698|
| 7782| CLARK|7839|
| 7698| BLAKE|7839|
| 7876| ADAMS|7788|
| 7566| JONES|7839|
| 7369| SMITH|7902|
| 7839|  KING|null|
| 7788| SCOTT|7566|
| 7654|MARTIN|7698|
+-----+------+----+



In [0]:
df8.alias("e").join(df8.alias("m"), col("e.MGR") == col("m.EMPNO"), "leftOuter").select(col("e.EMPNO").alias("Employee_id"), col("e.ENAME").alias("Employee_name"), col("e.MGR").alias("Manager_id"), col("m.ENAME").alias("Manager_Name")).show()

+-----------+-------------+----------+------------+
|Employee_id|Employee_name|Manager_id|Manager_Name|
+-----------+-------------+----------+------------+
|       7499|        ALLEN|      7698|       BLAKE|
|       7902|         FORD|      7566|       JONES|
|       7521|         WARD|      7698|       BLAKE|
|       7934|       MILLER|      7782|       CLARK|
|       7900|        JAMES|      7698|       BLAKE|
|       7844|       TURNER|      7698|       BLAKE|
|       7782|        CLARK|      7839|        KING|
|       7698|        BLAKE|      7839|        KING|
|       7876|        ADAMS|      7788|       SCOTT|
|       7566|        JONES|      7839|        KING|
|       7369|        SMITH|      7902|        FORD|
|       7839|         KING|      null|        null|
|       7788|        SCOTT|      7566|       JONES|
|       7654|       MARTIN|      7698|       BLAKE|
+-----------+-------------+----------+------------+



SELECTING EMPLOYEE NAME STARTING WITH S

In [0]:
df7.filter(upper(col("ENAME")).like('S%')).show()

+-----+-------+-----+----+----+----+------+----------+------------+
|ENAME|    JOB|EMPNO| MGR| SAL|COMM|DEPTNO|  HIREDATE|total_salary|
+-----+-------+-----+----+----+----+------+----------+------------+
|SMITH|  CLERK| 7369|7902| 800|   0|    20|1980-12-17|         800|
|SCOTT|ANALYST| 7788|7566|3000|   0|    20|1987-04-19|        3000|
+-----+-------+-----+----+----+----+------+----------+------------+



In [0]:
df7.where(upper(col("ENAME")).like('S%')).show()

+-----+-------+-----+----+----+----+------+----------+------------+
|ENAME|    JOB|EMPNO| MGR| SAL|COMM|DEPTNO|  HIREDATE|total_salary|
+-----+-------+-----+----+----+----+------+----------+------------+
|SMITH|  CLERK| 7369|7902| 800|   0|    20|1980-12-17|         800|
|SCOTT|ANALYST| 7788|7566|3000|   0|    20|1987-04-19|        3000|
+-----+-------+-----+----+----+----+------+----------+------------+



In [0]:
df7.summary()

Out[40]: DataFrame[summary: string, ENAME: string, JOB: string, EMPNO: string, MGR: string, SAL: string, COMM: string, DEPTNO: string, total_salary: string]

Employee name having salary greater than 2000

In [0]:
df7.select("ENAME", "total_salary").where(col("total_salary") > 2000).show()

+------+------------+
| ENAME|total_salary|
+------+------------+
|  FORD|        3020|
| CLARK|        2450|
| BLAKE|        2850|
| JONES|        2995|
|  KING|        5000|
| SCOTT|        3000|
|MARTIN|        2650|
+------+------------+



Clerks having salary greater than 2000

In [0]:
df7.dtypes

Out[53]: [('ENAME', 'string'),
 ('JOB', 'string'),
 ('EMPNO', 'int'),
 ('MGR', 'int'),
 ('SAL', 'int'),
 ('COMM', 'int'),
 ('DEPTNO', 'int'),
 ('HIREDATE', 'date'),
 ('total_salary', 'int')]

In [0]:
df7.select("ENAME", "JOB", "total_salary").where((col("total_salary") > 2000) & (col("JOB").isin("CLERK", "ANALYST"))).show()

+-----+-------+------------+
|ENAME|    JOB|total_salary|
+-----+-------+------------+
| FORD|ANALYST|        3020|
|SCOTT|ANALYST|        3000|
+-----+-------+------------+



LIST ALL JOB POSITIONS

In [0]:
df7.select(col("JOB")).distinct().show()

+---------+
|      JOB|
+---------+
|  ANALYST|
| SALESMAN|
|    CLERK|
|  MANAGER|
|PRESIDENT|
+---------+



Change datatype of total_salary

In [0]:
from pyspark.sql.types import LongType

In [0]:
df9 = df7.withColumn("total_salary", col("total_salary").cast(LongType()))

In [0]:
df9.show()

+------+---------+-----+----+----+----+------+----------+------------+
| ENAME|      JOB|EMPNO| MGR| SAL|COMM|DEPTNO|  HIREDATE|total_salary|
+------+---------+-----+----+----+----+------+----------+------------+
| ALLEN| SALESMAN| 7499|7698|1600| 300|    30|1981-02-20|        1900|
|  FORD|  ANALYST| 7902|7566|3000|  20|  1000|1981-12-03|        3020|
|  WARD| SALESMAN| 7521|7698|1250| 500|    30|1981-02-22|        1750|
|MILLER|    CLERK| 7934|7782|1300|  10|  1000|1982-01-23|        1310|
| JAMES|    CLERK| 7900|7698| 950|  30|  1000|1981-12-03|         980|
|TURNER| SALESMAN| 7844|7698|1500|   0|    30|1981-09-08|        1500|
| CLARK|  MANAGER| 7782|7839|2450|   0|    10|1981-06-09|        2450|
| BLAKE|  MANAGER| 7698|7839|2850|   0|    30|1981-05-01|        2850|
| ADAMS|    CLERK| 7876|7788|1100|  20|  1000|1987-05-23|        1120|
| JONES|  MANAGER| 7566|7839|2975|  20|  1000|1981-04-02|        2995|
| SMITH|    CLERK| 7369|7902| 800|   0|    20|1980-12-17|         800|
|  KIN

To find max salary, min salary, total salary, number of employees for each department

In [0]:
df7.select(col("DEPTNO"), col("total_salary")).groupBy(col("DEPTNO")).agg(max(col("total_salary")).alias("Maximum sal"), min(col("total_salary")).alias("Minimum sal"), avg(col("total_salary")).alias("Average sal"), count(col("total_salary")).alias("No of employees")).show()

+------+-----------+-----------+-----------+---------------+
|DEPTNO|Maximum sal|Minimum sal|Average sal|No of employees|
+------+-----------+-----------+-----------+---------------+
|    20|       3000|        800|     1900.0|              2|
|  1000|       3020|        980|     1885.0|              5|
|    10|       5000|       2450|     3725.0|              2|
|    30|       2850|       1500|     2130.0|              5|
+------+-----------+-----------+-----------+---------------+

